In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import category_encoders as ce
from sklearn import preprocessing
import numpy as np
import random

import json

class MTGAI(nn.Module):
    
    def __init__(self, Dsize):
        super(MTGAI, self).__init__()
        self.Embed1 = nn.Embedding(Dsize, 10)
        self.LSTM1 = nn.GRU(10, 500, 30) # input size, hidden size, num of layers to use
        self.Drop1 = nn.Dropout(.5)
        self.Drop2 = nn.Dropout(.5)
        self.Lin1 = nn.Linear(500, Dsize)
        self.Embed1.weight.data.uniform_(-.1, .1)
        self.Lin1.bias.data.fill_(0)
        self.Lin1.weight.data.uniform_(-.1, .1)
        
    def forward(self, x, h, drop):
        x = self.Embed1(x)
        if drop == True:
            x = self.Drop1(x)
        out, h = self.LSTM1(x, h)
        if drop == True:
            out = self.Drop2(out)
        decoded = self.Lin1(out.view(out.size(0)*out.size(1), out.size(2)))
        return decoded.view(out.size(0), out.size(1), decoded.size(1)), h
        
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return Variable(weight.new(30, batch_size, 500).zero_())
    

Dict = []
DataArray = []
TrueDict = {}

with open('AllSets.json') as json_data:
    data = json.load(json_data)
    
    
def get_batch(source, i, evaluation=False):
    seq_len = min(10, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    data = data.cuda()
    target = target.cuda()
    return data, target


for i, dataz in enumerate(data):
    print(dataz)
    for z, datazz in enumerate(data[dataz]['cards']):
        #print(list(dataz))
        #Name = dataz['name']
        #Ftext = dataz['flavorText']
        #print(Name)
        #print(Ftext)
        Rtext = []
        if 'text' in datazz:
            Rtext = datazz['text']
            DataArray.append(Rtext)
            for word in Rtext.split():
                if word not in Dict:
                    Dict.append(word)
        #print(Rtext)
        #print(len(Dict))
    
#print(Dict)
DataArray = ' '.join(DataArray)
DataArray = DataArray.split()
DataTensor = torch.zeros(int(len(DataArray)/10), 10, dtype=torch.long, device='cuda')
print(len(DataArray))
for i in range(9):
    DataArray.pop()
for i in range(len(Dict)):
    TrueDict[Dict[i]] = i
with torch.no_grad():    
    for i in range(int(len(DataArray)/10)):
        for x in range(10):
            DataTensor[i][x] = TrueDict[DataArray[i*10+x]]
#EncData = [TrueDict[e] for e in Dict if e in TrueDict]
#for i in range(686):
    #for z in range(10):
        #DataTensor[i][z] = DataArray[i*10+z]

AI = MTGAI(len(Dict))
AI = AI.cuda()
LossF1 = nn.CrossEntropyLoss()
OptimF1 = optim.Adam(AI.parameters(), lr=.001)
hidden = AI.init_hidden(10)
print(hidden)
bestloss = 0
epochs = 10
for epoch in range(epochs):
    totalloss = 0
    for i in range(94202):
        AI.train()
        dataz, targets = get_batch(DataTensor, i)
        hidden = Variable(hidden.data)
        OptimF1.zero_grad()
        predictions, hidden = AI(dataz, hidden, True)
        loss = LossF1(predictions.view(-1, len(Dict)), targets)
        if i % 1000 == 0:
            print("Trained on ",i*10, " words.")
        loss.backward()
        totalloss+= loss.item()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(AI.parameters(), .25)

        OptimF1.step()
        #total_loss += len(data) * loss.data
    print("Total loss this epoch:", totalloss)
    if bestloss > totalloss or bestloss==0:
        bestloss = totalloss
        print("New best loss! Val: ", bestloss)
        torch.save(AI.state_dict(), "./MTGAIbckup.model.pth")
#print(TrueDict)
print("Done with training")


10E
2ED
3ED
4ED
5DN
5ED
6ED
7ED
8ED
9ED
A25
AER
AKH
ALA
ALL
ANA
APC
ARB
ARC
ARN
ATH
ATQ
AVR
BBD
BFZ
BNG
BOK
BRB
BTD
C13
C14
C15
C16
C17
C18
CED
CEI
CHK
CHR
CM1
CM2
CMA
CMD
CN2
CNS
CON
CP1
CP2
CP3
CSP
CST
DD1
DD2
DDC
DDD
DDE
DDF
DDG
DDH
DDI
DDJ
DDK
DDL
DDM
DDN
DDO
DDP
DDQ
DDR
DDS
DDT
DDU
DGM
DIS
DKA
DKM
DOM
DPA
DRB
DRK
DST
DTK
DVD
E01
E02
EMA
EMN
EVE
EVG
EXO
EXP
F01
F02
F03
F04
F05
F06
F07
F08
F09
F10
F11
F12
F13
F14
F15
F16
F17
F18
FBB
FEM
FNM
FRF
FUT
G00
G01
G02
G03
G04
G05
G06
G07
G08
G09
G10
G11
G17
G18
G99
GK1
GK2
GNT
GPT
GRN
GS1
GTC
GVL
H09
H17
HHO
HML
HOP
HOU
HTR
HTR17
ICE
IMA
INV
ISD
ITP
J12
J13
J14
J15
J16
J17
J18
JGP
JOU
JUD
JVC
KLD
KTK
L12
L13
L14
L15
L16
L17
LEA
LEB
LEG
LGN
LRW
M10
M11
M12
M13
M14
M15
M19
MBS
MD1
ME1
ME2
ME3
ME4
MED
MGB
MIR
MM2
MM3
MMA
MMQ
MOR
MP2
MPR
MPS
MRD
NEM
NPH
OARC
OC13
OC14
OC15
OC16
OC17
OC18
OCM1
OCMD
ODY
OE01
OGW
OHOP
OLGC
ONS
OPC2
OPCA
ORI
OVNT
P02
P03
P04
P05
P06
P07
P08
P09
P10
P10E
P11
P15A
P2HG
PAER
PAKH
PAL00
PAL01
PAL02
PAL03
PAL04
PAL05
PA

Trained on  750000  words.
Trained on  760000  words.
Trained on  770000  words.
Trained on  780000  words.
Trained on  790000  words.
Trained on  800000  words.
Trained on  810000  words.
Trained on  820000  words.
Trained on  830000  words.
Trained on  840000  words.
Trained on  850000  words.
Trained on  860000  words.
Trained on  870000  words.
Trained on  880000  words.
Trained on  890000  words.
Trained on  900000  words.
Trained on  910000  words.
Trained on  920000  words.
Trained on  930000  words.
Trained on  940000  words.
Total loss this epoch: 747229.2713158131
Trained on  0  words.
Trained on  10000  words.
Trained on  20000  words.
Trained on  30000  words.
Trained on  40000  words.
Trained on  50000  words.
Trained on  60000  words.
Trained on  70000  words.
Trained on  80000  words.
Trained on  90000  words.
Trained on  100000  words.
Trained on  110000  words.
Trained on  120000  words.
Trained on  130000  words.
Trained on  140000  words.
Trained on  150000  words.
T

Trained on  910000  words.
Trained on  920000  words.
Trained on  930000  words.
Trained on  940000  words.
Total loss this epoch: 782799.3704531193
Trained on  0  words.
Trained on  10000  words.
Trained on  20000  words.
Trained on  30000  words.
Trained on  40000  words.
Trained on  50000  words.
Trained on  60000  words.
Trained on  70000  words.
Trained on  80000  words.
Trained on  90000  words.
Trained on  100000  words.
Trained on  110000  words.
Trained on  120000  words.
Trained on  130000  words.
Trained on  140000  words.
Trained on  150000  words.
Trained on  160000  words.
Trained on  170000  words.
Trained on  180000  words.
Trained on  190000  words.
Trained on  200000  words.
Trained on  210000  words.
Trained on  220000  words.
Trained on  230000  words.
Trained on  240000  words.
Trained on  250000  words.
Trained on  260000  words.
Trained on  270000  words.
Trained on  280000  words.
Trained on  290000  words.
Trained on  300000  words.
Trained on  310000  words.
T

Trained on  110000  words.
Trained on  120000  words.
Trained on  130000  words.
Trained on  140000  words.
Trained on  150000  words.
Trained on  160000  words.
Trained on  170000  words.
Trained on  180000  words.
Trained on  190000  words.
Trained on  200000  words.
Trained on  210000  words.
Trained on  220000  words.
Trained on  230000  words.
Trained on  240000  words.
Trained on  250000  words.
Trained on  260000  words.
Trained on  270000  words.
Trained on  280000  words.
Trained on  290000  words.
Trained on  300000  words.
Trained on  310000  words.
Trained on  320000  words.
Trained on  330000  words.
Trained on  340000  words.
Trained on  350000  words.
Trained on  360000  words.
Trained on  370000  words.
Trained on  380000  words.
Trained on  390000  words.
Trained on  400000  words.
Trained on  410000  words.
Trained on  420000  words.
Trained on  430000  words.
Trained on  440000  words.
Trained on  450000  words.
Trained on  460000  words.
Trained on  470000  words.
T

In [11]:
AI.load_state_dict(torch.load("./MTGAIbckup.model.pth"))
hidden = AI.init_hidden(1)
temperature = 20
num_words = 10
initword = Dict[random.randint(0, len(Dict))]
input = Variable(torch.LongTensor([[TrueDict[initword]]]).long())
input.data = input.data.cuda()
print(initword, '', end='')
for i in range(num_words):
    output, hidden = AI(input, hidden, False)
    word_weights = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    input.data.fill_(word_idx)
    for searchword, searchid in TrueDict.items():
        if searchid == word_idx:
            word = searchword


    print(word + ' ', end='')
print("Done")

Woolly Shapeshifter Well Afterlife thirteen card: Forest: Marauding Dirge Karona ({4}{U}{U}, Done
